In [22]:
from faker import Faker
import random
import datetime
import pandas as pd

# Inicialize o Faker
fake = Faker()

# Lista de nomes de tênis famosos
tennis_names = {"Nike Air Max":300, "Adidas Stan Smith":250, "Converse Chuck Taylor":200, "Vans Old Skool":190, 
                "Reebok Classic Leather":220, "Puma Suede":210, "New Balance 574":240, "Asics Gel-Kayano":230, 
                "Brooks Ghost":180, "Mizuno Wave Rider":270}

store_names = ["Sneaker Haven", "Footwear Emporium", "Kicks Kingdom", "Shoe Sanctuary", "Sole Central"]

# Número de linhas de dados que você quer gerar
num_rows = 500

# Lista para armazenar os dados gerados
data = []

# Gerar dados para cada linha
for _ in range(num_rows):
    # Calcular a data mínima permitida
    min_date = datetime.datetime.today() - datetime.timedelta(days=90)

    # Gerar uma data dentro dos últimos 3 meses
    created_at = fake.date_between(start_date=min_date, end_date='now')
    product = random.choice(list(tennis_names.keys()))
    store_name = random.choice(store_names)
    price = tennis_names[product] + round(random.uniform(-15,15),2) # Preço aleatório entre 100 e 300
    percentage_discount = round(random.uniform(0, 30), 2)/100  # Desconto percentual aleatório entre 0 e 50
    total_discount = round(price * (percentage_discount), 2)  # Calcula o desconto total

    data.append([created_at, product, store_name, price, percentage_discount, total_discount])


In [25]:
df = pd.DataFrame(data, columns=['created_at', 'product', 'store_name', 'price', 'percentage_discount', 'total_discount'])
df.to_csv('data.csv', index=False)

In [29]:
import numpy as np

def levenshtein_distance(s1, s2):
    m = len(s1)
    n = len(s2)
    dp = np.zeros((m+1, n+1))

    for i in range(m+1):
        dp[i][0] = i
    for j in range(n+1):
        dp[0][j] = j

    for i in range(1, m+1):
        for j in range(1, n+1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j],        # Deletion
                                   dp[i][j-1],        # Insertion
                                   dp[i-1][j-1])     # Substitution
    return dp[m][n]

def match_product_names(name1, name2, threshold):
    distance = levenshtein_distance(name1.lower(), name2.lower())
    max_length = max(len(name1), len(name2))
    similarity = 1 - (distance / max_length)
    if similarity >= threshold:
        return True
    else:
        return False

# Exemplo de uso:
name1 = "Puma Sued"
name2 = "Puma Suede"
threshold = 0.7  # Ajuste conforme necessário
if match_product_names(name1, name2, threshold):
    print("Os nomes dos produtos correspondem.")
else:
    print("Os nomes dos produtos não correspondem.")

Os nomes dos produtos correspondem.


In [44]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import cosine_similarity
from tensorflow.keras.optimizers import Adam

# Função para calcular a distância de cosseno entre vetores
def cosine_distance(vectors):
    x, y = vectors
    x = tf.nn.l2_normalize(x, axis=1)
    y = tf.nn.l2_normalize(y, axis=1)
    return 1 - cosine_similarity(x, y)  # Negativo da distância de cosseno para obter similaridade

# Função para construir a arquitetura da rede siamês
def siamese_model(input_shape):
    input_1 = Input(shape=input_shape)
    input_2 = Input(shape=input_shape)

    lstm_layer = LSTM(128)
    encoded_1 = lstm_layer(input_1)
    encoded_2 = lstm_layer(input_2)

    distance = Lambda(cosine_distance)([encoded_1, encoded_2])

    model = Model(inputs=[input_1, input_2], outputs=distance)
    return model

# Dados de entrada: nomes de produtos
name1 = "Puma Suede Classic"
name2 = "Nike Air max"

# Vetorizando os nomes de produtos (representação simples para este exemplo)
max_length = 20
input_dim = 1
input_data_1 = np.array([[ord(char) for char in name1.ljust(max_length)]])  # Padding para o comprimento máximo
input_data_2 = np.array([[ord(char) for char in name2.ljust(max_length)]])  # Padding para o comprimento máximo

# Normalizando os dados
input_data_1 = input_data_1 / 255.0
input_data_2 = input_data_2 / 255.0

# Definindo os rótulos (0 para não similar, 1 para similar)
labels = np.array([1])

# Definindo a forma dos dados de entrada
input_shape = (max_length, input_dim)

# Construindo e compilando o modelo
model = siamese_model(input_shape)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Avaliação
similarity_score = model.predict([input_data_1, input_data_2])
print("Similaridade entre os nomes dos produtos:", similarity_score)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
Similaridade entre os nomes dos produtos: [1.9942555]
